In [61]:
import numpy as np
import pandas as pd

import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import time
from bs4 import BeautifulSoup as BSoup

from env import github_token, github_username

### Create function to generate URLs to scrape
- need a LIST in te format of "gocodeup/codeup-setup-script"
- search criteria: https://github.com/search?q=oceanography&type=repositories
- page 2 is: https://github.com/search?p=2&q=oceanography&type=Repositories
- type parameter changes capitalization....?p=1 exists, so just use that


In [2]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}
p=1
url = f'https://github.com/search?p={p}&q=oceanography&type=Repositories'
url

'https://github.com/search?p=1&q=oceanography&type=Repositories'

In [3]:
#grab the first page
response = requests.get(url, headers=headers)
response.status_code

200

In [7]:
soup = BSoup(response.text)

**now try and get all the repo urls on the page**
- the repo search results page has each repo in its own li
- in that list there are two divs.  The one we want is div class='mt-n1 flex-auto>
- then another div class="d-flex"
- then another div class="f4 text-normal"
- then:

a: 

class="v-align-middle" 

data-hydro-click= *lots of text*

data-hydro-click-hmac= *alphanumeric string* 

href="/rabernat/intro_to_physical_oceanography">rabernat/intro_to_physical_
<em>oceanography</em> /a


In [14]:
len(soup.find_all('div', {'class':'f4 text-normal'})) 
#GOOD - it gets 10 (which is our num results per page)

10

In [15]:
#let's look at the first 2
soup.find_all('div', {'class':'f4 text-normal'})[0:2]

[<div class="f4 text-normal">
 <a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"oceanography","result_position":1,"click_id":41701566,"result":{"id":41701566,"global_relay_id":"MDEwOlJlcG9zaXRvcnk0MTcwMTU2Ng==","model_name":"Repository","url":"https://github.com/rabernat/intro_to_physical_oceanography"},"originating_url":"https://github.com/search?p=1&amp;q=oceanography&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="2429d6b0fa54733ee57c2aedde47122dabd955a1ee2d1f8e6f5bb61c633959b3" href="/rabernat/intro_to_physical_oceanography">rabernat/intro_to_physical_<em>oceanography</em></a>
 </div>,
 <div class="f4 text-normal">
 <a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"oceanography","result_position":2,"click_id":570570,"result":{"id":570570,"global_relay_id":"MDEwOlJlcG9zaXRvcnk1NzA1NzA=","model_name":"Reposito

In [18]:
#seems to grab what we want, so now we just need the href of each one
type(soup.find_all('div', {'class':'f4 text-normal'})[0]) #tag

bs4.element.Tag

In [24]:
soup.find_all('div', {'class':'f4 text-normal'})[0].select_one('a')

<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"oceanography","result_position":1,"click_id":41701566,"result":{"id":41701566,"global_relay_id":"MDEwOlJlcG9zaXRvcnk0MTcwMTU2Ng==","model_name":"Repository","url":"https://github.com/rabernat/intro_to_physical_oceanography"},"originating_url":"https://github.com/search?p=1&amp;q=oceanography&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="2429d6b0fa54733ee57c2aedde47122dabd955a1ee2d1f8e6f5bb61c633959b3" href="/rabernat/intro_to_physical_oceanography">rabernat/intro_to_physical_<em>oceanography</em></a>

In [25]:
type(soup.find_all('div', {'class':'f4 text-normal'})[0].select_one('a'))

bs4.element.Tag

In [26]:
soup.find_all('div', {'class':'f4 text-normal'})[0].select_one('a')['href']

'/rabernat/intro_to_physical_oceanography'

so that ^^ works.  but maybe I can go straight to the a tag?  let's see if that class is unique

In [28]:
len(soup.find_all('a',{'class':'v-align-middle'}))
#YAY - only 10!

10

In [31]:
#Okay, so the shorter way is:
soup.find_all('a',{'class':'v-align-middle'})[0]['href']

'/rabernat/intro_to_physical_oceanography'

**Now figure out next page** - seems straight forward, here's the anchor tag:

class="next_page" 

rel="next" 

href="/search?p=2&amp;q=oceanography&amp;type=Repositories">Next /a>

In [33]:
soup.find('a',{'class':'next_page'})

<a class="next_page" href="/search?p=2&amp;q=oceanography&amp;type=Repositories" rel="next">Next</a>

In [34]:
soup.find('a',{'class':'next_page'})['href']

'/search?p=2&q=oceanography&type=Repositories'

**Test logic for when next page doesn't exist:**

In [37]:
#this is the last of many pages
response2 = requests.get('https://github.com/search?p=90&q=oceanography&type=Repositories',headers=headers)
response2.status_code

200

In [38]:
soup2 = BSoup(response2.text)

In [43]:
if not soup2.find('a',{'class':'next_page'}): print('hi')

hi


**okay so we need to check if none prior to grabbing the href**

In [77]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

def get_repo_links(soup):
    '''
    Given the soup of a repo search page, get a list urls for all the repos in the results
    
    Output: Returns LIST of partial URLs
    Parameters: (R) soup: Beautiful Soup object of the page contents
    '''
    pg_repos = []
    #loop over all our 'a' tags
    for a in soup.find_all('a',{'class':'v-align-middle'}):
        #grab the href and strip first character >> append to repo list
        pg_repos.append(a['href'][1:])
    #return the list 
    return pg_repos
        
def get_oceanography_repos():
    '''
    Gets a list of oceanography repos from Github.  
    Must have your own github token and username in a local env file. 
    Output: Returns LIST of strings - with section of URL in format 'user/repo'
    '''
    #set starting point
    #we know there are multiple pages, so including query parameter 'p'
    p=1
    url = f'https://github.com/search?p={p}&q=oceanography&type=Repositories'
    
    #FIRST PAGE
    #get page info
    response = requests.get(url, headers=headers)
    #SOUP, there it is!
    soup = BSoup(response.text)
    
    #grab links for first page
    repo_list = get_repo_links(soup)
    
    #NEXT PAGE - 
    #set domain
    domain = 'https://github.com'
    #get first next page
    np_tag = soup.find('a',{'class':'next_page'})
    #IF that element exists, go to and repeat!
    while np_tag:
        #create url for next page
        url = domain + np_tag['href']
        
        #get new soup
        response = requests.get(url, headers=headers)
        soup = BSoup(response.text)
        
        #add new links to our list
        repo_list += get_repo_links(soup)
        
        #break loop if we have > X links
        if len(repo_list) >= 300: break
        
        #grab next page
        np_tag = soup.find('a',{'class':'next_page'})
        time.sleep(7)
        
    return repo_list
    
def create_REPOS():
    filename = 'repos.csv'
    #see if file exists
    if os.path.isfile(filename):
        df = pd.read_csv(filename,header=None)
        return df.iloc[:0].tolist()
    else:
        #get new list of repos
        mylist = get_oceanography_repos()
        #save to CSV
        pd.DataFrame(mylist).to_csv(filename,header=False,index=False)
        return mylist


In [69]:
testing = get_oceanography_repos()
testing[-5:]

https://github.com/search?p=2&q=oceanography&type=Repositories
200
20
<a class="next_page" href="/search?p=3&amp;q=oceanography&amp;type=Repositories" rel="next">Next</a>
https://github.com/search?p=3&q=oceanography&type=Repositories
200
30
<a class="next_page" href="/search?p=4&amp;q=oceanography&amp;type=Repositories" rel="next">Next</a>
https://github.com/search?p=4&q=oceanography&type=Repositories
200
40
<a class="next_page" href="/search?p=5&amp;q=oceanography&amp;type=Repositories" rel="next">Next</a>
https://github.com/search?p=5&q=oceanography&type=Repositories
200
50
<a class="next_page" href="/search?p=6&amp;q=oceanography&amp;type=Repositories" rel="next">Next</a>
https://github.com/search?p=6&q=oceanography&type=Repositories
200
60
<a class="next_page" href="/search?p=7&amp;q=oceanography&amp;type=Repositories" rel="next">Next</a>
https://github.com/search?p=7&q=oceanography&type=Repositories
200
70
<a class="next_page" href="/search?p=8&amp;q=oceanography&amp;type=Reposito

['haloissufo/TOOLBOXES',
 'panmits86/thesis_1',
 'clase69/ocesat',
 'jbarthelotte/DFO-Moorings',
 'jklymak/Eos314']

In [50]:
len(testing)

90

In [75]:
pd.DataFrame(testing).to_csv('testing.csv',header=False,index=False)

In [76]:
pd.read_csv('testing.csv',header=None)

,0
0,rabernat/intro_to_physical_oceanography
1,dankelley/oce
2,wafo-project/pywafo
3,ocefpaf/python4oceanographers
4,matplotlib/cmocean
...,...
305,haloissufo/TOOLBOXES
306,panmits86/thesis_1
307,clase69/ocesat
308,jbarthelotte/DFO-Moorings


After adding some troubleshooting code, it looks like github is limiting my scrapping as I get a 429 response code (too many requests), so I added a pause.

##### NOW TEST THIS!

In [78]:
myrepos = create_REPOS()
myrepos[0:5]

['rabernat/intro_to_physical_oceanography',
 'dankelley/oce',
 'wafo-project/pywafo',
 'ocefpaf/python4oceanographers',
 'matplotlib/cmocean']

In [79]:
len(myrepos)

300

### Functions for scraping repo data after URLs were generated

In [ ]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""


# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = []
 

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_contents = requests.get(get_readme_download_url(contents)).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)
